In [1]:
print("Hello Quoc Anh")

Hello Quoc Anh


In [2]:
from flask import Flask, render_template_string, request, redirect, url_for, session
import random
from collections import Counter
import threading
import time

In [3]:
class Card:
    def __init__(self, color):
        self.color = color
    def __repr__(self):
        return self.color

class Player:
    def __init__(self, name, hand=None, is_bot=False):
        self.name = name
        self.hand = hand or []
        self.is_bot = is_bot
        self.passed = False # Đã dừng chơi (hết nước đi)

class Board:
    def __init__(self, size):
        self.size = size
        self.rows = []
        for i in range(size):
            self.rows.append([None]*(size-i))
    def place_card(self, row, col, card):
        self.rows[row][col] = card
    def display(self):
        for i, row in enumerate(self.rows):
            print("  " * i + " ".join([str(c) if c else "." for c in row]))

In [4]:
def is_valid_move(board, row, col, card):
    if row == 0:
        return board.rows[row][col] is None
    left_below = board.rows[row-1][col]
    right_below = board.rows[row-1][col+1]
    if left_below is None or right_below is None:
        return False
    return card.color in [left_below.color, right_below.color]

In [5]:
def get_all_valid_moves(board, hand):
    moves = []
    for card in hand:
        for row in range(board.size):
            for col in range(len(board.rows[row])):
                if board.rows[row][col] is None:
                    if is_valid_move(board, row, col, card):
                        moves.append((card, row, col))
    return moves

In [6]:
def bot_choose_move(board, hand):
    if not hand:
        return None
    color_count = Counter([card.color for card in hand])
    valid_moves = get_all_valid_moves(board, hand)
    if not valid_moves:
        return None
    best_color = color_count.most_common(1)[0][0]
    best_moves = [move for move in valid_moves if move[0].color == best_color]
    if best_moves:
        return best_moves[0]
    return valid_moves[0]

In [7]:
# Hàm khởi tạo ván chơi mới
def init_game():
    colors = ["Đỏ", "Xanh", "Vàng", "Tím"]
    deck = [Card(color) for color in colors for _ in range(6)]
    random.shuffle(deck)
    board = Board(size=5)
    player = Player("Bạn", hand=[deck.pop() for _ in range(8)], is_bot=False)
    bot = Player("Bot", hand=[deck.pop() for _ in range(8)], is_bot=True)
    return board, player, bot

In [8]:
# Hàm chuyển đổi object -> dict để lưu vào session
def card_to_dict(card):
    return {'color': card.color}
def dict_to_card(card_dict):
    return Card(card_dict['color'])
def board_to_list(board):
    return [[c.color if c else None for c in row] for row in board.rows]
def list_to_board(board_list):
    size = len(board_list)
    board = Board(size)
    for i, row in enumerate(board_list):
        for j, color in enumerate(row):
            if color:
                board.rows[i][j] = Card(color)
    return board

In [9]:
TEMPLATE = '''
<html>
<head>
  <title>Penguin Party Web vs Bot</title>
  <style>
    body { font-family: sans-serif; }
    .board-row { margin-left: 20px; }
    .card { display: inline-block; width: 60px; height: 80px; border: 1px solid #ccc; margin: 2px; text-align: center; line-height: 80px; font-weight: bold;}
    .Đỏ { background: #f88;}
    .Xanh { background: #8ef;}
    .Vàng { background: #ff8;}
    .Tím { background: #c9f;}
    .empty { background: #fff;}
    .btn { margin: 2px; }
  </style>
</head>
<body>
  <h2>Penguin Party Web vs Bot</h2>
  <h3>Bàn chơi:</h3>
  {% set size = board|length %}
  {% for i in range(size) %}
    <div class="board-row" style="margin-left:{{ (size-i-1)*32 }}px;">
      {% for card in board[size-i-1] %}
        <span class="card {{card if card else 'empty'}}">{{ card if card else '' }}</span>
      {% endfor %}
    </div>
  {% endfor %}
  <hr>
  <h3>Bài của bạn (chọn lá và vị trí để đánh):</h3>
  <form method="POST">
    {% for idx in range(hand|length) %}
      <button class="btn" name="card_idx" value="{{idx}}">{{ hand[idx].color }}</button>
    {% endfor %}
    <br><br>
    <label>Hàng (0 = dưới cùng): <input name="row" type="number" min="0" max="4" value="0"></label>
    <label>Cột: <input name="col" type="number" min="0" max="4" value="0"></label>
  </form>
  <hr>
  <h4>Bài còn lại của Bot: {{bot_hand_count}}</h4>
  {% if msg %}<b>{{msg}}</b>{% endif %}
  <br>
  <form method="POST" action="/reset">
    <button>Chơi lại</button>
  </form>
</body>
</html>
'''


In [10]:
app = Flask(__name__)
app.secret_key = "QuocAnhtapcode" 

@app.route("/", methods=["GET", "POST"])
def game():
    # Lấy trạng thái từ session hoặc khởi tạo mới
    if "board" not in session:
        board, player, bot = init_game()
        session['board'] = board_to_list(board)
        session['player_hand'] = [card_to_dict(c) for c in player.hand]
        session['bot_hand'] = [card_to_dict(c) for c in bot.hand]
        session['player_passed'] = False
        session['bot_passed'] = False
        msg = ""
    else:
        board = list_to_board(session['board'])
        player = Player("Bạn", hand=[dict_to_card(c) for c in session['player_hand']])
        bot = Player("Bot", hand=[dict_to_card(c) for c in session['bot_hand']], is_bot=True)
        player.passed = session['player_passed']
        bot.passed = session['bot_passed']
        msg = ""

    # Xử lý nước đi của người chơi
    if request.method == "POST" and "card_idx" in request.form:
        try:
            card_idx = int(request.form["card_idx"])
            row = int(request.form["row"])
            col = int(request.form["col"])
            card = player.hand[card_idx]
            if board.rows[row][col] is not None:
                msg = "Vị trí này đã có bài rồi!"
            elif is_valid_move(board, row, col, card):
                board.place_card(row, col, card)
                player.hand.pop(card_idx)
                # Lượt của bot sau khi bạn đi
                valid_bot_moves = get_all_valid_moves(board, bot.hand)
                if valid_bot_moves:
                    bot_move = bot_choose_move(board, bot.hand)
                    if bot_move:
                        bot_card, bot_row, bot_col = bot_move
                        board.place_card(bot_row, bot_col, bot_card)
                        bot.hand.remove(bot_card)
                        msg = f"Bạn đặt {card.color} ({row},{col}). Bot đặt {bot_card.color} ({bot_row},{bot_col})"
                else:
                    session['bot_passed'] = True
                    msg = "Bot không còn nước đi hợp lệ, dừng chơi!"
            else:
                msg = "Nước đi không hợp lệ theo luật Penguin Party!"
        except Exception as e:
            msg = "Có lỗi: " + str(e)

    # Kiểm tra kết thúc game
    if not player.hand:
        msg += " Bạn đã thắng! "
    elif not bot.hand:
        msg += " Bot đã thắng! "
    elif not get_all_valid_moves(board, player.hand):
        session['player_passed'] = True
        msg += " Bạn không còn nước đi hợp lệ, dừng chơi! "

    # Lưu trạng thái lại
    session['board'] = board_to_list(board)
    session['player_hand'] = [card_to_dict(c) for c in player.hand]
    session['bot_hand'] = [card_to_dict(c) for c in bot.hand]
    session['player_passed'] = player.passed
    session['bot_passed'] = bot.passed

    return render_template_string(TEMPLATE, board=board_to_list(board),
                                 hand=player.hand, bot_hand_count=len(bot.hand), msg=msg)

@app.route("/reset", methods=["POST"])
def reset():
    session.clear()
    return redirect(url_for('game'))


In [ ]:
def run_app():
    app.run()
    
threading.Thread(target=run_app).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jul/2025 09:27:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2025 09:27:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Jul/2025 09:27:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2025 09:28:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2025 09:28:22] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2025 09:28:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2025 09:28:54] "POST / HTTP/1.1" 200 -
